# Scope

In [1]:
# When Python compiles the body of the function, it decides that b is a local variable because it is assigned within the function

b = 'b'

def f1(a):
    print(a)
    print(b)
    b = 1

try:
    f1('a')
except Exception as e:
    print(e)


a
local variable 'b' referenced before assignment


In [2]:
# to fix this we can use global declaration

b = 'b'

def f2(a):
    global b
    print(a)
    print(b)
    b = 1

f2('a')

a
b


# Closure

Closures is what we get when functions capture variables defined outside of their bodies.  
  
**Closure** is a function *f* with an extended scope that encompases variables referenced in the body of *f* that are not global variables or local variables of *f*  
  
Closures allow us to preserve state while using functional approach (see [running average](../tasks/running_average.ipynb))

# Variable Lookup Logic

- if there is a *global x* declaration, *x* comes from and is assigned to module global variable *x* (python does not have a program global scope, only module global scopes)
- if there is a *nonlocal x* declaration, *x* comes from and is assigned to local variable *x* of the nearest surrounding function where *x* is defined
- if *x* is a parameter or is assigned a value in the function body, the *x* is the local variable
- if *x* is referenced but is not assigned and is not a parameter:
    - *x* will be looked up in the local scopes of the surrounding function bodies (nonlocal scopes)
    - if not found in surrounding scopes, it will be read from the module global scope
    - if not found in the global scope, it will be read from \_\_builtins__.__dict__.

# Decorators

A decorator is a callable that takes another function as an argument, 
extending the behavior of that function without explicitly modifying that function

https://dev.to/apcelent/python-decorator-tutorial-with-example-529f#

```
@decorator
def target():
    ...
```
the same as:

```
target = decorator(target)
```

Decorator function is executed when decorated function is defined (at import time as soon as the module is imported).

## Function decorator

In [3]:
def handleException(func):
    def wrapper(*args):
        try:
            return func(*args)
        except TypeError:
            print('Type error')
        except ZeroDivisionError:
            print('Zero division error')
        except:
            print('Some error')
    return wrapper

@handleException
def causeError(n):
    return 1/n

# @handleException - shortcut for:
# causeError = handleException(causeError)

print(causeError(1))
causeError(0)

1.0
Zero division error


In [4]:
from functools import wraps

def my_decorator(func):
  @wraps(func)              # to preserve func name and docstring
  def wrapper(*args, **kwargs):
    # do something before
    result = func(*args, **kwargs)
    # do something after
    return result
  return wrapper

@my_decorator  
def func():
  return 'function output'
  
# equivalent to
func = my_decorator(func)

func()

'function output'

## Class decorator

In [5]:
from functools import update_wrapper

class Count():
  def __init__(self, func):
    update_wrapper(self, func)          # for correct docstring and name of the function
    self.func = func
    self.cnt = 0
  def __call__(self, *args, **kwargs):
    self.cnt += 1
    print(f"Current count: {self.cnt}")
    result = self.func(*args, **kwargs)
    return result

@Count
def fib(n):
  if n < 2:
    return n
  else:
    return fib(n-1) + fib(n-2)

fib(4)      # 9 calls bcz we are not caching values and calculate many of them several times during recursive execution

Current count: 1
Current count: 2
Current count: 3
Current count: 4
Current count: 5
Current count: 6
Current count: 7
Current count: 8
Current count: 9


3

In [6]:
# for caching we can use lru_cache from functools

from functools import wraps, lru_cache

@lru_cache(maxsize=None)
@Count
def fib(n):
  if n < 2:
    return n
  else:
    return fib(n-1) + fib(n-2)

fib(4)

Current count: 1
Current count: 2
Current count: 3
Current count: 4
Current count: 5


3

# Parametrized decorator

In [7]:
registry = []
print('registry ->', registry)

def register(func):
    print(f'Running register({func})')
    registry.append(func)
    return func

@register
def f1():
    print('Running f1()')

f1()
print('registry ->', registry)

registry -> []
Running register(<function f1 at 0x7fc89179d940>)
Running f1()
registry -> [<function f1 at 0x7fc89179d940>]


In [8]:
registry = set()

def register(active=True):
    def wrapper(func):
        print(f'Running register(active={active}) -> decorate({func})')
        registry.add(func) if active else registry.discard(func)
        return func
    return wrapper

@register(active=False)
def f1():
    print('running f1()')

@register()
def f2():
    print('running f2()')

def f3():
    print('running f3()')
register()(f3)

print(registry)

Running register(active=False) -> decorate(<function f1 at 0x7fc89179dd30>)
Running register(active=True) -> decorate(<function f2 at 0x7fc89179d940>)
Running register(active=True) -> decorate(<function f3 at 0x7fc891feca60>)
{<function f2 at 0x7fc89179d940>, <function f3 at 0x7fc891feca60>}


In [13]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    def outer_wrapper(func):
        def inner_wrapper(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return inner_wrapper
    return outer_wrapper

@clock()
def snooze(seconds):
    time.sleep(seconds)

for _ in range(3):
    snooze(0.1)

print('-----------------------')

@clock('{name}: {elapsed:.6f}s')
def snooze(seconds):
    time.sleep(seconds)

for _ in range(3):
    snooze(0.1)

[0.10020330] snooze(0.1) -> None
[0.10018390] snooze(0.1) -> None
[0.10018970] snooze(0.1) -> None
-----------------------
snooze: 0.100186s
snooze: 0.100207s
snooze: 0.100167s


In [14]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}] {name}({args}) -> {result}'

class clock:
    def __init__(self, fmt=DEFAULT_FMT):
        self.fmt = fmt

    def __call__(self, func):
        def wrapper(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(self.fmt.format(**locals()))
            return _result
        return wrapper

@clock()
def snooze(seconds):
    time.sleep(seconds)

for _ in range(3):
    snooze(0.1)

print('-----------------------')

@clock('{name}: {elapsed:.6f}s')
def snooze(seconds):
    time.sleep(seconds)

for _ in range(3):
    snooze(0.1)

[0.10026790] snooze(0.1) -> None
[0.10017260] snooze(0.1) -> None
[0.10017060] snooze(0.1) -> None
-----------------------
snooze: 0.100199s
snooze: 0.100175s
snooze: 0.100162s
